In [1]:
#Task Functions (used to facilitate tasks)
import urllib. request
import time
import glob, os
import json

#create a data folder to store all the raw data
if not os.path.exists('data'):
    os.mkdir('data')
    print("Directory data created")   

# pull course catalog pages
def catalog():
    #list of urls from the MIT course catalog
    file = open('./00_urls.txt')
    lines = file.readlines()
    url_list = [line.rstrip() for line in lines]
    
    #urls_list = open('./00_urls.txt').readlines()
    #print(url_list)
    
    def pull(url):
        response = urllib.request.urlopen(url).read()
        data = response.decode('utf-8')
        return data
    def store(data,file):
        # Create path:
        path = os.path.join('./data', file)

        # Create a file a file
        page = os.open(path, os.O_RDWR|os.O_CREAT )

        # Write tp the file
        os.write(page, data.encode())

        # Close the file
        os.close(page)

        print('wrote file: ' + file)
    
    def combine():

        if os.path.exists('combo.txt'):
            os.remove('combo.txt')
            print("combo.txt removed") 
        page = os.open('combo.txt', os.O_RDWR|os.O_CREAT )
        os.close(page)
        print("combo.txt created") 

        with open('combo.txt','w') as outfile:
            for file in glob.glob("./data/*.html"):
                with open(file) as infile:
                    outfile.write(infile.read())

    #loop through all urls
    for url in url_list:
        try:
            #get page name
            file=url.split("/")[-1]
            data=pull(url)
            store(data,file)        
            print(f'Pulled: {url}')
            print('--- waiting ---')
            time.sleep(15)
        except:
            print(f'Not found: {url}')
        
        

In [3]:
def combine():
        
    if os.path.exists('combo.txt'):
        os.remove('combo.txt')
        print("combo.txt removed") 
    page = os.open('combo.txt', os.O_RDWR|os.O_CREAT )
    os.close(page)
    print("combo.txt created") 
        
    with open('combo.txt','w') as outfile:
        for file in glob.glob("./data/*.html"):
            with open(file) as infile:
                outfile.write(infile.read())


In [5]:
combine()

combo.txt removed
combo.txt created


In [31]:
url_list = os.open('./00_urls.txt', os.O_RDONLY)
# Close the file
#    os.close(page)

In [75]:
file = open('./00_urls.txt')
url_list = file.readlines()
lines = [line.rstrip() for line in lines]

In [46]:
# Open a file
fd = os.open( './00_urls.txt', os.O_RDWR|os.O_CREAT )
lines = readlines(fd)
lines = [line.rstrip() for line in lines]

NameError: name 'readlines' is not defined

In [ ]:
index = url.rfind('/') + 1
#call pull function

file = url[index:]
#call store function

print('pulled: ' + file)
print('--- waiting ---')
time.sleep(15)

In [99]:
def combine():
    with open('combo.txt','w') as outfile:
           for file in glob.glob("./data/*.html"):
                with open(file) as infile:
                    outfile.write(infile.read())

In [7]:
from bs4 import BeautifulSoup
def store_json(data,file):
       with open(file, 'w', encoding='utf-8') as f:
           json.dump(data, f, ensure_ascii=False, indent=4)
           print('wrote file: ' + file)

In [10]:
def titles():
    from bs4 import BeautifulSoup
    def store_json(data,file):
           with open(file, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=4)
                print('wrote file: ' + file)

    #Open and read the large html file generated by combine()
    with open('combo.txt') as file:
        html=file.read()

    #the following replaces new line and carriage return char
    html = html.replace('\n', ' ').replace('\r', '')
    #the following create an html parser
    soup = BeautifulSoup(html, "html.parser")
    results = soup.find_all('h3')
    titles = []

    # tag inner text
    for item in results:
        titles.append(item.text)
    store_json(titles, 'titles.json')

In [8]:
with open('combo.txt') as file:
    file=file.read()

In [11]:
titles()

wrote file: titles.json


In [15]:
def clean():
    #complete helper function definition below
    def store_json(data,file):
           with open(file, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=4)
                print('wrote file: ' + file)
                
    with open('titles.json') as file:
        titles = json.load(file)
        # remove punctuation/numbers
        for index, title in enumerate(titles):
            punctuation= '''!()-[]{};:'"\,<>./?@#$%^&*_~1234567890'''
            translationTable= str.maketrans("","",punctuation)
            clean = title.translate(translationTable)
            titles[index] = clean

        # remove one character words
        for index, title in enumerate(titles):
            clean = ' '.join( [word for word in title.split() if len(word)>1] )
            titles[index] = clean

        store_json(titles, 'titles_clean.json')

In [16]:
clean()

wrote file: titles_clean.json


In [19]:
def count_words():
    from collections import Counter
    def store_json(data,file):
           with open(file, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=4)
                print('wrote file: ' + file)
    with open('titles_clean.json') as file:
        titles = json.load(file)
        words = []

        # extract words and flatten
        for title in titles:
            words.extend(title.split())

        # count word frequency
        counts = Counter(words)
        store_json(counts, 'words.json')

In [20]:
count_words()

wrote file: words.json


In [4]:
file = open(r'C:\Users\cni4\Documents\MIT_Data_Science\Projects\SenseMakingDataPipeline/00_urls.txt')

In [6]:
file.read()


'http://student.mit.edu/catalog/m1a.html\nhttp://student.mit.edu/catalog/m1b.html\nhttp://student.mit.edu/catalog/m1c.html\nhttp://student.mit.edu/catalog/m2a.html\nhttp://student.mit.edu/catalog/m2b.html\nhttp://student.mit.edu/catalog/m2c.html\nhttp://student.mit.edu/catalog/m3a.html\nhttp://student.mit.edu/catalog/m3b.html\nhttp://student.mit.edu/catalog/m4a.html\nhttp://student.mit.edu/catalog/m4b.html\nhttp://student.mit.edu/catalog/m4c.html\nhttp://student.mit.edu/catalog/m4d.html\nhttp://student.mit.edu/catalog/m4e.html\nhttp://student.mit.edu/catalog/m4f.html\nhttp://student.mit.edu/catalog/m4g.html\nhttp://student.mit.edu/catalog/m5a.html\nhttp://student.mit.edu/catalog/m5b.html\nhttp://student.mit.edu/catalog/m6a.html\nhttp://student.mit.edu/catalog/m6b.html\nhttp://student.mit.edu/catalog/m6c.html\nhttp://student.mit.edu/catalog/m7a.html\nhttp://student.mit.edu/catalog/m8a.html\nhttp://student.mit.edu/catalog/m8b.html\nhttp://student.mit.edu/catalog/m9a.html\nhttp://student.

In [7]:
    #list of urls from the MIT course catalog
    file = open('./00_urls.txt')
    lines = file.readlines()
    url_list = [line.rstrip() for line in lines]

In [8]:
url_list

['http://student.mit.edu/catalog/m1a.html',
 'http://student.mit.edu/catalog/m1b.html',
 'http://student.mit.edu/catalog/m1c.html',
 'http://student.mit.edu/catalog/m2a.html',
 'http://student.mit.edu/catalog/m2b.html',
 'http://student.mit.edu/catalog/m2c.html',
 'http://student.mit.edu/catalog/m3a.html',
 'http://student.mit.edu/catalog/m3b.html',
 'http://student.mit.edu/catalog/m4a.html',
 'http://student.mit.edu/catalog/m4b.html',
 'http://student.mit.edu/catalog/m4c.html',
 'http://student.mit.edu/catalog/m4d.html',
 'http://student.mit.edu/catalog/m4e.html',
 'http://student.mit.edu/catalog/m4f.html',
 'http://student.mit.edu/catalog/m4g.html',
 'http://student.mit.edu/catalog/m5a.html',
 'http://student.mit.edu/catalog/m5b.html',
 'http://student.mit.edu/catalog/m6a.html',
 'http://student.mit.edu/catalog/m6b.html',
 'http://student.mit.edu/catalog/m6c.html',
 'http://student.mit.edu/catalog/m7a.html',
 'http://student.mit.edu/catalog/m8a.html',
 'http://student.mit.edu/catalog

In [2]:

print("Path at terminal when executing this file")
print(os.getcwd() + "\n")

print("This file path, relative to os.getcwd()")
print(__file__ + "\n")

print("This file full path (following symlinks)")
full_path = os.path.realpath(__file__)
print(full_path + "\n")

print("This file directory and name")
path, filename = os.path.split(full_path)
print(path + ' --> ' + filename + "\n")

print("This file directory only")
print(os.path.dirname(full_path))

Path at terminal when executing this file
C:\Users\cni4\Documents\MIT_Data_Science\Projects\SenseMakingDataPipeline

This file path, relative to os.getcwd()


NameError: name '__file__' is not defined